In [1]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex

import import_ipynb

import data_utils as data


tt = gp.Model('IIITB Exams Timetable')

31
aasmaan.gupta@iiitb.ac.in
samarjeet.wankhade@iiitb.ac.in
Tanmay.Jain015@iiitb.ac.in
Shreyan.Gupta@iiitb.ac.in
Shubham.Agarwal@iiitb.ac.in
Nimish.Khandeparkar@iiitb.ac.in
Vidhu.Arora@iiitb.ac.in
Mayank.Sharma@iiitb.ac.in
Ayush.Singh@iiitb.ac.in
Rithik.Bansal@iiitb.ac.in
Yash.Bansal513@iiitb.ac.in
Kushal.Jenamani@iiitb.ac.in
Madhav.Patil@iiitb.ac.in
Ashashree.Sarma@iiitb.ac.in
Patelrishita.Dipak@iiitb.ac.in
Pranita.Ganguly@iiitb.ac.in
Shindedev.Hemravi@iiitb.ac.in
Suhan.Roy@iiitb.ac.in
AadilMohammad.Husain@iiitb.ac.in
fitriana.dewi@iiitb.ac.in
Ritish.Shrirao@iiitb.ac.in
Trupti.Khodwe@iiitb.ac.in
Harsh.Dhruv@iiitb.ac.in
Madhav.Girdhar@iiitb.ac.in
Divyam.Sareen@iiitb.ac.in
Prateek.Rath@iiitb.ac.in
Swetha.Murali@iiitb.ac.in
Daksh.Rajesh@iiitb.ac.in
Rutul.Patel@iiitb.ac.in
Nathan.Verghese@iiitb.ac.in
Ananthula.Reddy@iiitb.ac.in
Varnit.Mittal@iiitb.ac.in
Siddeshwar.Kagatikar@iiitb.ac.in
Manda.Kausthubh@iiitb.ac.in
Nikita.Kiran@iiitb.ac.in
Siddharth.Maramreddy@iiitb.ac.in
Srivatsa.Tarun@iii

## Make one binary variable for each room-course-timeslot-day combination
$$
E_{crtd} =
\begin{cases} 
1 & \text{if a course $c$ exam is scheduled in room $r$ at time $t$ on day $d$,} \\
0 & \text{Otherwise.}
\end{cases}
$$

In [2]:
def exam_length(course:int):
    return 120

def exam_slots(course: int):
    # return (data.get_exam_duration(course)//15)

    return int(exam_length(course)//15)
#Dhruv's part:

# session_spans = {s: begin_end_pairs(session_length(s)) for s in range(n_sessions)}
# print(data.begin_end_pairs(exam_slots(1)))
exam_spans = {c: data.begin_end_pairs(exam_slots(c)) for c in range(data.n_courses)}

E = tt.addVars(
    [
        (c ,b ,e, d)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="E",
)

C = tt.addVars(
    [
        
        (c,r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.INTEGER,
    lb=0,
    ub=230,
    name="C",
)



## Hard Constraints

In [3]:
def signum(x):
    if x>0:
        return 1
    return 0

def covering_sessions(s:int, t:int):
    # Return the list of begin-end pairs for session 's' overlapping with time 't'
    return [(b,e) for b,e in exam_spans[s] if  data.overlap(b,e,t)]

In [4]:
# Exactly one slot for each course
# sum of enrolments in courses assigned to a room <= room capacity
# no clash for any student
# all slots for a course on any day are contiguous
# exam lengths for each course

In [5]:


# Constraint 1: Each exam is scheduled exactly once
tt.addConstrs(
    (E.sum(c, '*', '*', '*') == 1 for c in range(data.n_courses)),
    name="one_exam_per_course"
)

tt.update()

In [6]:
# Constraint 2: Room capacity constraint C[c, r]*E[c, b, e, d] <= room_capacity[r] for all rooms

# tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
#                           for c in range(data.n_courses)
#                           for b,e in covering_sessions(c, t)) <= data.get_room_capacity(r)
#               for d in range(data.n_days)
#               for r in range(data.n_rooms)
#               for t in range(data.n_times)),
#               name = "room_capacity_constraint"
#              )

P = tt.addVars(
    [
        (c, r, b, e, d)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.INTEGER,
    name = 'P',
    lb = 0, 
    ub = 230,
)


for c,r,b,e,d in P:
    room_capacity = data.get_room_capacity(r)
    tt.addConstr(P[c,r,b,e,d] >=0, name = 'P1_constraint')
    tt.addConstr(P[c,r,b,e,d] <= room_capacity*E[c,b,e,d], name = 'P2_constraint')
    tt.addConstr(P[c,r,b,e,d] <= C[c,r]*(1-E[c,b,e,d]), name = 'P3_constraint')
    tt.addConstr(P[c,r,b,e,d] >= (C[c,r] - room_capacity)*(1-E[c,b,e,d]), name = 'P4_constraint')


tt.update()

tt.addConstrs(
    (gp.quicksum(P[c, r, b, e, d] 
                 for r in range(data.n_rooms) 
                 for b, e in covering_sessions(c, t)) <= data.get_room_capacity(r)
     for d in range(data.n_days) 
     for c in range(data.n_courses) 
     for t in range(data.n_times)),
    name='room_capacity_constraint'
)

tt.update()


In [7]:
# Constraint 3: All students for a course should write the exam 

# tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
#                           for r in range(data.n_rooms)
#                           for b,e in covering_sessions(c, t)) <= data.get_course_strength(c)
#               for d in range(data.n_days)
#               for c in range(data.n_courses)
#               for t in range(data.n_times)),
#               name = 'course_enrollment_constraint'
#              )

# # Constraint 3.1: All students for a course should write the exam
# tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
#                            for d in range(data.n_days)
#                           for r in range(data.n_rooms)
#                           for t in range(data.n_times)
#                           for b,e in covering_sessions(c, t)) == data.get_course_strength(c)
#               for c in range(data.n_courses)),
#               name = 'course_enrollment_constraint'
#              )

# Constraint 3.1: All students for a course should write the exam

tt.addConstrs(
    gp.quicksum(C[c,r] for r in range(data.n_rooms)) == data.get_course_strength(c)
    for c in range(data.n_courses)
)



# tt.addConstrs(
#     (gp.quicksum(P[c, r, b, e, d] 
#                  for r in range(data.n_rooms) 
#                  for b, e in covering_sessions(c, t)) == data.get_course_strength(c)
#      for d in range(data.n_days) 
#      for c in range(data.n_courses) 
#      for t in range(data.n_times)),
#     name='course_enrollment_constraint'
# )

tt.update()


In [8]:
# # Constraint 4: Student clash constraint

# # for each student, for each day, for each time slot, at most one exam
# # sigma Ecbed <= 1 for courses enrolled by student s


tt.addConstrs((gp.quicksum(E[c,b,e,d] for c in data.get_courses_student(s) 
                for b,e in covering_sessions(c, t)) <= 1
                for s in range(data.n_students)
                for d in range(data.n_days)
                for t in range(data.n_times)),
              name = 'student_clash_constraint'
             )

# tt.update()

{(0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 18): <gurobi.Constr *Awaiting 

In [9]:
# Constraint 5: No course can have more than 3 rooms alloted


M = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'M',
)

D1 = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'D1',
)

D2 = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'D2',
)



for c,r in M:
    room_capacity = data.get_room_capacity(r)
    tt.addConstr(M[c,r] <=1, name = 'M1_constraint')
    tt.addConstr(M[c,r] <=C[c,r], name = 'M2_constraint')
    tt.addConstr(M[c,r] >=D1[c,r], name = 'M3_constraint')
    tt.addConstr(M[c,r] >=C[c,r]-(room_capacity*(1-D2[c,r])), name = 'M4_constraint')
    tt.addConstr(D1[c,r]+D2[c,r] == 1, name = 'M5_constraint')

# tt.addConstrs(
#     (
#         # gp.quicksum(signum(C[c, r]) for r in range(data.n_rooms)) <= 3
#         gp.quicksum(min(1, C[c, r]) for r in range(data.n_rooms)) <= 3
#         for c in range(data.n_courses)
#     ),
#     name="course_room_constraint",
# )

tt.addConstrs(
    (
        gp.quicksum(M[c, r] for r in range(data.n_rooms)) <= 3
        for c in range(data.n_courses)
    ),
    name="course_room_constraint",
)

tt.update()

## Soft Constraints

In [10]:
# preferred time slots for exams

## Objective Function

In [11]:
#objective function: minimize sum of E[c,b,e,d] for all c,b,e,d

objective = E.sum()

tt.setObjective(objective, sense=GRB.MINIMIZE)

tt.update()


In [12]:
# Maximize Gaps between exams for a student / batch
tt.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.1 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2615578 - for non-commercial use only - registered to sa___@iiitb.ac.in
Optimize a model with 605554 rows, 211352 columns and 7159256 nonzeros
Model fingerprint: 0x57773455
Model has 395560 quadratic constraints
Variable types: 0 continuous, 211352 integer (11774 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 2e+02]
  QRHS range       [6e+00, 2e+02]
Presolve added 299944 rows and 0 columns
Presolve removed 0 rows and 1798 columns
Presolve time: 2.46s
Presolved: 905498 rows, 209554 columns, 8065922 nonzeros
Variable types: 0 continuous,

In [13]:
E_sol = tt.getAttr('x', E)
C_sol = tt.getAttr('x', C)
M_sol = tt.getAttr('x', M)
D1_sol = tt.getAttr('x', D1)
D2_sol = tt.getAttr('x', D2)

#I want to know my optimal solution
# print(type(E_sol))
# print(len(E_sol))
# print(E_sol)
# print(C_sol)
# print(M_sol)
# print(D1_sol)
# print(D2_sol)

#How many solutions are there?
# print(tt.objVal)

#Get all the values from E_sol whose value is 1
solution=[]
for i in E_sol:
    if E_sol[i] == 1:
        solution.append(i)

for i in solution:
    print(i)

(0, 21, 29, 1)
(1, 21, 29, 2)
(2, 0, 8, 3)
(3, 0, 8, 1)
(4, 3, 11, 4)
(5, 0, 8, 2)
(6, 29, 37, 4)
(7, 29, 37, 1)
(8, 26, 34, 1)
(9, 0, 8, 0)
(10, 0, 8, 2)
(11, 9, 17, 0)
(12, 0, 8, 1)
(13, 0, 8, 3)
(14, 0, 8, 0)
(15, 0, 8, 2)
(16, 29, 37, 2)
(17, 29, 37, 1)
(18, 3, 11, 3)
(19, 0, 8, 1)
(20, 0, 8, 0)
(21, 29, 37, 4)
(22, 29, 37, 3)
(23, 3, 11, 4)
(24, 29, 37, 0)
(25, 29, 37, 0)
(26, 29, 37, 0)
(27, 9, 17, 0)
(28, 29, 37, 1)
(29, 12, 20, 4)
(30, 0, 8, 0)
(31, 29, 37, 0)
(32, 29, 37, 3)
(33, 29, 37, 1)
(34, 23, 31, 4)
(35, 27, 35, 1)
(36, 0, 8, 0)
(37, 29, 37, 3)
(38, 29, 37, 4)
(39, 0, 8, 3)
(40, 29, 37, 2)
(41, 29, 37, 4)
(42, 26, 34, 0)
(43, 29, 37, 3)
(44, 0, 8, 0)
(45, 29, 37, 2)
(46, 0, 8, 1)
(47, 29, 37, 3)
(48, 29, 37, 1)
(49, 26, 34, 0)
(50, 21, 29, 2)
(51, 29, 37, 4)
(52, 29, 37, 3)
(53, 0, 8, 2)
(54, 0, 8, 0)
(55, 29, 37, 4)
(56, 0, 8, 2)
(57, 0, 8, 1)


In [14]:
print(C_sol)
solution_rooms=[]
for i in C_sol:
    if C_sol[i] > 0:
        solution_rooms.append(i)

for i in solution_rooms:
    print(i)

{(0, 0): 14.0, (0, 1): -0.0, (0, 2): -0.0, (0, 3): -0.0, (0, 4): -0.0, (0, 5): -0.0, (0, 6): -0.0, (0, 7): -0.0, (0, 8): -0.0, (0, 9): -0.0, (0, 10): -0.0, (0, 11): -0.0, (0, 12): -0.0, (0, 13): -0.0, (0, 14): -0.0, (0, 15): -0.0, (0, 16): -0.0, (0, 17): -0.0, (0, 18): -0.0, (0, 19): -0.0, (0, 20): -0.0, (0, 21): -0.0, (0, 22): -0.0, (0, 23): -0.0, (0, 24): -0.0, (0, 25): -0.0, (0, 26): -0.0, (0, 27): -0.0, (0, 28): -0.0, (0, 29): -0.0, (0, 30): 6.0, (1, 0): -0.0, (1, 1): 95.0, (1, 2): -0.0, (1, 3): -0.0, (1, 4): -0.0, (1, 5): -0.0, (1, 6): -0.0, (1, 7): -0.0, (1, 8): -0.0, (1, 9): -0.0, (1, 10): -0.0, (1, 11): -0.0, (1, 12): -0.0, (1, 13): -0.0, (1, 14): -0.0, (1, 15): -0.0, (1, 16): -0.0, (1, 17): -0.0, (1, 18): -0.0, (1, 19): -0.0, (1, 20): -0.0, (1, 21): -0.0, (1, 22): -0.0, (1, 23): -0.0, (1, 24): -0.0, (1, 25): -0.0, (1, 26): -0.0, (1, 27): -0.0, (1, 28): -0.0, (1, 29): -0.0, (1, 30): 6.0, (2, 0): 51.0, (2, 1): -0.0, (2, 2): -0.0, (2, 3): -0.0, (2, 4): -0.0, (2, 5): -0.0, (2, 6):

In [20]:
for c,b,e,d in solution:
    l = []
    for(r) in range(data.n_rooms):
        if C_sol[c,r] > 0:
            l.append((data.get_room_num(r), C_sol[c,r]))
    print(f"Course {data.course_id_code[c]} has {data.get_course_strength(c)} and is scheduled in slot {data.times[b]} to {data.times[e]} on day {data.days[d]} in room {l}")

Course AIM 704 has 20 and is scheduled in slot 14:00 to 16:00 on day Tue in room [('R102', 14.0), ('R-310', 6.0)]
Course AIM 825A has 101 and is scheduled in slot 14:00 to 16:00 on day Wed in room [('R103', 95.0), ('R-310', 6.0)]
Course AIM 825B has 57 and is scheduled in slot 08:00 to 10:00 on day Thu in room [('R102', 51.0), ('R-310', 6.0)]
Course AIM 829 has 132 and is scheduled in slot 08:00 to 10:00 on day Tue in room [('R103', 126.0), ('R-310', 6.0)]
Course AIM 831 has 175 and is scheduled in slot 08:45 to 10:45 on day Fri in room [('R103', 169.0), ('R-310', 6.0)]
Course AIM 832 has 93 and is scheduled in slot 08:00 to 10:00 on day Wed in room [('R103', 87.0), ('R-310', 6.0)]
Course CSE 816 has 146 and is scheduled in slot 16:00 to 18:00 on day Fri in room [('R103', 140.0), ('R-310', 6.0)]
Course DAS 101A has 96 and is scheduled in slot 16:00 to 18:00 on day Tue in room [('R103', 90.0), ('R-310', 6.0)]
Course DAS 101B has 95 and is scheduled in slot 15:15 to 17:15 on day Tue in r

In [29]:
for s in range (data.n_students):
    l=[]
    course_list=data.get_courses_student(s)
    for c,b,e,d in solution:
        if E_sol[c,b,e,d] == 1 and c in course_list:
            l.append((data.course_id_code[c], data.times[b], data.times[e], data.days[d]))
    
    print(f"Student {s+1} has exams for courses {l}")

Student 1 has exams for courses [('AIM 704', '14:00', '16:00', 'Tue'), ('AIM 829', '08:00', '10:00', 'Tue'), ('CSE 819', '14:00', '16:00', 'Wed')]
Student 2 has exams for courses [('AIM 704', '14:00', '16:00', 'Tue'), ('DHS 308', '16:00', '18:00', 'Thu'), ('DHS 314', '16:00', '18:00', 'Fri'), ('DHS 315', '08:00', '10:00', 'Thu')]
Student 3 has exams for courses [('AIM 704', '14:00', '16:00', 'Tue'), ('AIM 829', '08:00', '10:00', 'Tue'), ('AIM 832', '08:00', '10:00', 'Wed')]
Student 4 has exams for courses [('AIM 704', '14:00', '16:00', 'Tue'), ('AIM 831', '08:45', '10:45', 'Fri'), ('CSE 824', '08:00', '10:00', 'Mon')]
Student 5 has exams for courses [('AIM 704', '14:00', '16:00', 'Tue'), ('AIM 829', '08:00', '10:00', 'Tue'), ('DHS 315', '08:00', '10:00', 'Thu')]
Student 6 has exams for courses [('AIM 704', '14:00', '16:00', 'Tue'), ('AIM 829', '08:00', '10:00', 'Tue')]
Student 7 has exams for courses [('AIM 704', '14:00', '16:00', 'Tue'), ('AIM 831', '08:45', '10:45', 'Fri'), ('CSE 824